# Prototype

#### Data processing

In [112]:
import pandas as pd 


In [113]:
reddit_df = pd.read_csv('../data/reddit_dataset.csv')
reddit_df.head()

,Unnamed: 0,id,comment,score
0,0,1,Dumbass liberals. You’d let anyone fuck you o...,0.979
1,1,2,"Yeah ok, shut the fuck up bitch. It’s Bernie o...",0.958
2,2,3,"fuck you, you're just pretending to be blocked...",0.938
3,3,4,"""He's a borderline rapist and that's just what...",0.938
4,4,5,"If you stick your dick in crazy, the crazy sti...",0.938


In [114]:
len(reddit_df)

6000

In [115]:
reddit_df.columns

Index(['Unnamed: 0', 'id', 'comment', 'score'], dtype='object')

In [116]:
#drop unnamed:0 colunm 

In [117]:
reddit_df.drop(['Unnamed: 0'], axis=1, inplace=True)

Drop rows with no-longer availble (deleted or removed) comments.

In [118]:
for index, row in reddit_df.iterrows():
    if row['comment'] == '[removed]' or row['comment'] == '[deleted]' or row['comment'] == 'this comment is no longer availble':
        reddit_df.drop(index, axis=0, inplace=True)

Checking if I miss any deleted or removed comment

In [119]:
for index, row in reddit_df.iterrows():
    if row['comment'] == '[removed]' or row['comment'] == '[deleted]':
        print('you missed something')

It confirmed that all rows with no comments are removed. 

In [120]:
len(reddit_df)

5721

In total of 245 rowsa are deleted. It reduce the number of data but it is something that I cannot do anything about it. 

Now, aplit dateframe into two dataframe : Train, test, and validation data.

In [121]:
#reset indexing of datafram 
reddit_df.reset_index(drop=True, inplace=True)

### Text cleaning 

Text cleaning process involves removing unncessary words such as stopwords, and remove punctuations. 

In [122]:
import string 
from nltk.corpus import stopwords

In [136]:
def clean_text(text):
    #tokenize text 
    tokens = text.split()
    #remove punctuation 
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return str(tokens)

In [137]:
t = "I am saying, you cant be too generous as in, you shouldn't be too generous..."
c = clean_text(t)
print(c)

['saying', 'cant', 'generous', 'shouldnt', 'generous']


itterate through each text in dataframe and clean each text

In [142]:
for index, row in reddit_df.iterrows():
    reddit_df.iloc[index, 1] = clean_text(row['comment'])


In [143]:
#shuffle rows 
reddit_df = reddit_df.sample(frac=1).reset_index(drop=True)

In [144]:
reddit_df.head()

,id,comment,score
0,1641,"['What', 'difference', 'make', 'They', 'used',...",0.146
1,1720,"['Im', 'fan', 'conservatism', 'lets', 'blindly...",0.125
2,38,"['Dude', 'youre', 'fucking', 'Emma', 'Watson',...",0.854
3,4692,"['Bulgaria', 'Croatia', 'Czech', 'Republic', '...",-0.292
4,5250,"['Im', 'allergic', 'ive', 'got', 'valid', 'rea...",-0.396


In [145]:
train_df = reddit_df.iloc[:4604,:]
val_df = reddit_df.iloc[4605:,:]

In [146]:
train_df.to_csv('train_test_data.csv')
val_df.to_csv('val_data.csv')

clean up training data